In [ ]:
import os
import tensorflow as tf

# Disable all GPUs by setting the environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Disable all GPUs

# Alternatively, using TensorFlow to set visible devices to an empty list for 'GPU'
tf.config.set_visible_devices([], 'GPU')

# Verify that TensorFlow is not using any GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs are still available:", gpus)
else:
    print("No GPUs available, TensorFlow is using only CPU.")


No GPUs available, TensorFlow is using only CPU.


: 

In [ ]:
# Import necessary libraries
import os
from datetime import datetime
import rasterio
from tqdm import tqdm
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import json
from tensorflow import keras
from leaf_utils import *
from leaf_form_processing import *
from leaf_parameter_calculate import *
from circle_parameter_calculate import *
from circle_processing import *
from leaf_predition import *
from imaging_circle_updated import *
from imaging_leaf_updated import *
from mrcnn import model as modellib
from mrcnn.config import Config
from single_slope_calculation import *
from single_area_prediction import *
from pathlib import Path

# Define matrices F1 and F4
F1 = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]])
F4 = np.array([
    [1, 1, 1, 1, -4, 1, 1, 1, 1],
    [1, 1, -3.6, -3.2, -3, -3.2, -3.6, 1, 1],
    [1, -3.6, -2.8, -2.2, -2, -2.2, -2.8, -3.6, 1],
    [1, -3.2, -2.2, -1.4, -1, -1.4, -2.2, -3.2, 1],
    [-4, -3, -2, -1, 0, -1, -2, -3, -4],
    [1, -3.2, -2.2, -1.4, -1, -1.4, -2.2, -3.2, 1],
    [1, -3.6, -2.8, -2.2, -2, -2.2, -2.8, -3.6, 1],
    [1, 1, -3.6, -3.2, -3, -3.2, -3.6, 1, 1],
    [1, 1, 1, 1, -4, 1, 1, 1, 1]
])

# Define necessary paths and constants \\192.168.91.83\Register\farmerspace\fs24_zkr_tellow_frauenhofer\proc\orthomosaic\rededge-p\9m\20240801.tif
ORTHO_FOLDER = "//192.168.91.83/Register/farmerspace/fs24_zkr_tellow_frauenhofer/proc/orthomosaic/rededge-p/9m/"
ortho_file = "20240819.tif"
DEM_FOLDER = "C:/temp_disease_quantification/fs24_zkr_tellow_frauenhofer/proc/dem/rededge-p/9m/"
SLOPE_FOLDER= "C:/temp_disease_quantification/fs24_zkr_tellow_frauenhofer/proc/a2c/rededge-p/9m"
AREA_FOLDER= "C:/temp_disease_quantification/fs24_zkr_tellow_frauenhofer/proc/area_coeficient/rededge-p/9m"
shape_plot_path="//192.168.91.83/Register/farmerspace/fs24_zkr_tellow_frauenhofer/raw/gis/PrecisionBeet_fieldtrial_aktuell/PrecisionBeet_fieldtrial"
shape_plot_file = "shape_mini_wgs_automatic_scoring.gpkg"
RESULTS_FOLDER_ROOT = "results"
design_miniplot_path ="qgis_labels.xlsx"

pipeline_sources_path = "//192.168.91.193/Dokumente/ifz_daten/COBRI/SOURCES/R/pipeline_sources"
path_model="//192.168.91.193/Dokumente/ifz_daten/COBRI/co19_cobri/proc/ml_modeling/classifiers/"
INSTANCE_FOLDER="C:/temp_disease_quantification/fs24_zkr_tellow_frauenhofer/proc/leaf_shape/rededge-p/9m"
rdata_file = "Multispectral_indices.csv"
pipeline_path = "result_dict.json"
MRCNN_ROOT = "C:/Users/barreto/OneDrive - Institut für Zuckerrübenforschung/project_mask_rcnn/leaf_seg_SHRINATH/shrinath/Envs/models"
# MRCNN_MODEL_DIR = "C:/Users/barreto/AppData/Local/anaconda3/envs/abc/Lib/site-packages/mrcnn/models/selected_weights/05_100plus0"
MRCNN_MODEL_NAME = "mask_rcnn_sugarbeat_0050.h5"
MRCNN_MODEL_PATH = "C:/Users/barreto/OneDrive - Institut für Zuckerrübenforschung/project_mask_rcnn/leaf_seg_SHRINATH/shrinath/Envs/models/selected_weights/05_100plus0/mask_rcnn_sugarbeat_0050.h5"
# MRCNN_MODEL_PATH = os.path.join(MODEL_DIR, MRCNN_MODEL_NAME)
date_id = datetime.now().strftime("%Y%m%d")


# Load the design miniplot
design_miniplot = pd.read_excel(design_miniplot_path, sheet_name="data")

# Load model pipeline configuration
model_label = "wf_022"
with open(pipeline_path, 'r') as f:
    pipeline = json.load(f)
coef_names = pipeline['coef']
class_output = pipeline["class_character"]

# Set constants and parameters
UTMzone = 32
input_image_size = 2400  # Size of images that cut from the big orthomosaic to predict the leaves 
rad = 0.05               # Radius for circle instance
n_ins = 100              # Number of instances
n_ins_test = n_ins * 1.5 # Instances for sampling (LEAF form)
th_prediction = 20       # Minimal percentage of vegetation to sample
min_pix_pred = 30        # Minimal number of pixels of vegetation to sample
th_erosion = 10          # Erosion activation threshold
IMAGING = True           # Option to print images
PRINT_SHAPE_SEL_LEAF = True
FOLDER_DEM_FEATURES = DEM_FOLDER
MAX_AREA_LEAF = 225      # Maximum leaf area in cm2
VERSION_PARAMETERS = "0001"

# Check necessary files
def check_files_exist(paths):
    """Check if all the files in the list exist."""
    for path in paths:
        if not os.path.exists(path):
            raise FileNotFoundError(f"{path} does not exist, PLEASE CHECK FILES/PATHS")
    print("All crucial files checked")

# Define file paths

id_file = ortho_file[:8]
INSTANCE_FORM = "BOTH"
DI_THRESHOLD = 0.50  
bands = ["BLUE", "GREEN", "RED", "REDEDGE","NIR"]
file_id_ortho = ortho_file[:8]
ortho_file_path = os.path.join(ORTHO_FOLDER, ortho_file)
dem_file_path = os.path.join(DEM_FOLDER, ortho_file)
shape_plot_full_path = 'shape_mini_wgs_automatic_scoring.gpkg' #os.path.join(shape_plot_path, shape_plot_file)
cnn_model_path = os.path.join(pipeline['model_path'][0], pipeline['model_list'][0])

# Check if necessary files exist
check_files_exist([ortho_file_path, dem_file_path, cnn_model_path])

# Create results folder if it doesn't exist
RESULTS_FOLDER_ROOT = os.path.join(RESULTS_FOLDER_ROOT, date_id)
os.makedirs(RESULTS_FOLDER_ROOT, exist_ok=True)
print(f"Results folder created at: {RESULTS_FOLDER_ROOT}")

# Define additional paths
a2c_path_list = [os.path.join(DEM_FOLDER, "a2c", ortho_file)]
area_coeficient_path_list = [os.path.join(DEM_FOLDER, "area_coeficient", ortho_file)]

# LEAF PROCESSING
INSTANCE_FORM = "LEAF"  # Set to "LEAF" or "BOTH"
shape_leaf_path = os.path.join(INSTANCE_FOLDER, f"{file_id_ortho}.gpkg")

# Load the .RData file
Index_List = pd.read_csv(rdata_file)

# Initialize variables
values = None
file = 1

# PROCESSING

# Load sample ortho DEM
print(f"Loading ortho file: {ortho_file_path}")
M1 = rasterio.open(ortho_file_path)

# Load features
slope_path = os.path.join(SLOPE_FOLDER, ortho_file)
if not os.path.exists(slope_path):
    single_slope_calculation(DEM_FOLDER, ortho_file, slope_path, epsg_utm="EPSG:32632", epsg_wgs84="EPSG:4326", chunk_size=1000)
if not os.path.exists(slope_path):
    print("Slope file missing. Please calculate slope!")

area_path = os.path.join(AREA_FOLDER, ortho_file)
if not os.path.exists(area_path):
    single_area_prediction(DEM_FOLDER, ortho_file, area_path)
if not os.path.exists(area_path):
    print("Area coefficient file missing. Please calculate area coefficient!")

with rasterio.open(ortho_file_path) as src:
    profile = src.profile

# Extract data from the ortho and DEM files
rmu_data_t = np.mean(real_res(ortho_file_path, UTMzone))
rmux_data_t = real_res(ortho_file_path, UTMzone)[0]
rmuy_data_t = real_res(ortho_file_path, UTMzone)[1]
D1 = rasterio.open(dem_file_path)
rde_data_t = np.mean(real_res(dem_file_path, UTMzone))

# Load pixel size and slope
pix_size = rasterio.open(area_path)
a2cam = rasterio.open(slope_path)

# Load leaf shape
shape_plot = gpd.read_file("shape_automatic_scoring.gpkg")
shape_plot = shape_plot.to_crs('EPSG:4326')
shape_plot['geometry'] = shape_plot['geometry'].apply(convert_multipolygon_to_polygon)

if 'plot_id' not in shape_plot.columns:
    shape_plot['plot_id'] = range(1, len(shape_plot) + 1)
plot_list_id = sorted(shape_plot['plot_id'])    
# Check and process LEAF instances
LEAF_PROCESSING = False
if INSTANCE_FORM in ["LEAF", "BOTH"]:
    if os.path.exists(shape_leaf_path):
        # Read the shapefile if it exists
        shape_leaf = gpd.read_file(shape_leaf_path)
        print("Shapefile read successfully.")
        LEAF_PROCESSING = True
    else:
        print('Loading the Mask R-CNN Model...')

        # Define configuration for Mask R-CNN
        class CocoSynthConfig(Config):
            NAME = "cocosynth_dataset"
            GPU_COUNT = 1
            IMAGES_PER_GPU = 1
            NUM_CLASSES = 1 + 1  # Background + 1 leaf type
            IMAGE_MIN_DIM = 2048
            IMAGE_MAX_DIM = 2048
            STEPS_PER_EPOCH = 2000
            VALIDATION_STEPS = 100
            BACKBONE = 'resnet101'
            RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)
            TRAIN_ROIS_PER_IMAGE = 32
            MAX_GT_INSTANCES = 50
            POST_NMS_ROIS_TRAINING = 500
            
            

        config = CocoSynthConfig()

        class InferenceConfig(CocoSynthConfig):
            GPU_COUNT = 1
            IMAGES_PER_GPU = 1
            IMAGE_MIN_DIM = 4096
            IMAGE_MAX_DIM = 4096
            DETECTION_MIN_CONFIDENCE = 0.5
            DETECTION_MAX_INSTANCES = 4000
            POST_NMS_ROIS_INFERENCE =  10000

        inference_config = InferenceConfig()
        model = modellib.MaskRCNN(mode="inference", model_dir=MRCNN_ROOT, config=inference_config)
        model.load_weights(MRCNN_MODEL_PATH, by_name=True)
        
        print('Model loaded successfully.')
        leaf_predictions(ORTHO_FOLDER, ortho_file, model, shape_leaf_path,input_image_size)
        shape_leaf = gpd.read_file(shape_leaf_path)
        LEAF_PROCESSING = True

INSTANCE_FORM = 'BOTH'

# Process LEAF and CIRCLE instances if LEAF_PROCESSING is enabled
if LEAF_PROCESSING:
    model = keras.models.load_model(cnn_model_path)
    multi_poly, labels = [], []
    
    for P in tqdm(range(226,len(shape_plot))):
        plot_id_temp = plot_list_id[P]
        # Crop plot
        shape_temp = shape_plot[shape_plot['plot_id'] == plot_id_temp]
        try:
            S1, S1_transform, S1_bounds = read_raster_window(shape_temp, ortho_file_path)
        except Exception as e:
            print(f"Error cropping plot: {e}")
            continue
        
        if S1 is not None:
            # Create a dictionary to store parameters
                params = {
                    'S1': S1,
                    'plot_id_temp': plot_id_temp,
                    'RESULTS_FOLDER_ROOT': RESULTS_FOLDER_ROOT,
                    'INSTANCE_FORM': INSTANCE_FORM,
                    'shape_leaf': shape_leaf,
                    'shape_temp': shape_temp,
                    'ortho_file_path': ortho_file_path,
                    'area_path': area_path,
                    'rmux_data_t': rmux_data_t,
                    'rmuy_data_t': rmuy_data_t,
                    'n_ins_test': n_ins_test,
                    'MAX_AREA_LEAF': MAX_AREA_LEAF,
                    'bands': bands,
                    'pipeline': pipeline,
                    'model': model,
                    'class_output': class_output,
                    'min_pix_pred': min_pix_pred,
                    'th_erosion': th_erosion,
                    'F1': F1,
                    'slope_path': slope_path,
                    'profile': profile,
                    'n_ins': n_ins,
                    'DI_THRESHOLD': DI_THRESHOLD,
                    'UTMzone': UTMzone
                }

                # Process condition plot
                condition_plot_results = process_condition_plot(params)

                # Extract returned values from the dictionary
                LEAF_CONDITION = condition_plot_results['LEAF_CONDITION']
                leaf_mask_pred = condition_plot_results['mask_pred']
                leaf_instance_vector = condition_plot_results['instance_vector']
                leaf_cluster_vector = condition_plot_results['cluster_vector']
                size_vector = condition_plot_results['size_vector']
                slope_vector = condition_plot_results['slope_vector']
                T1_leaf = condition_plot_results['T1']
                n_ins_sample = condition_plot_results['n_ins_sample']
                RESULTS_FOLDER = condition_plot_results['RESULTS_FOLDER']
                folder_name_temp = condition_plot_results['folder_name_temp']
                final_sel_leaf_id = condition_plot_results['final_sel_leaf_id']
                T1_transform = condition_plot_results['T1_transform']
                output_leaf_shape_plot = condition_plot_results['output_leaf_shape_plot']
                rr_ed_leaf = condition_plot_results['rr_ed']

                # Generate and save leaf images
                imaging_leaf_updated(T1_leaf,RESULTS_FOLDER,rr_ed_leaf,leaf_cluster_vector,leaf_mask_pred,shape_temp,ortho_file_path)      

                # Update the parameter dictionary with additional values
                params.update({
                    'leaf_mask_pred': leaf_mask_pred,
                    'leaf_cluster_vector': leaf_cluster_vector.flatten(),
                    'leaf_instance_vector': leaf_instance_vector,
                    'size_vector': size_vector,
                    'slope_vector': slope_vector,
                    'n_ins_sample': n_ins_sample,
                    'T1_leaf': T1_leaf,
                    'shape_leaf': shape_leaf,
                    'RESULTS_FOLDER': RESULTS_FOLDER,
                    'folder_name_temp': folder_name_temp,
                    'plot_id_temp': plot_id_temp,
                    'final_sel_leaf_id': final_sel_leaf_id,
                    'output_leaf_shape_plot': output_leaf_shape_plot
                })

                # Calculate leaf parameters
                leaf_param_results = leaf_parameter_cal(params)

                # Extract returned values from the dictionary
                leaf_parameter = leaf_param_results['leaf_parameter']
                DECISION_PLOT_temp = leaf_param_results['DECISION_PLOT_temp']

                # Process circles
                circle_process_results = circle_processing(params)

                # Extract returned values from the dictionary
                circle_prediction = circle_process_results['mask_pred_circle']
                circle_cluster_vector = circle_process_results['cluster_vector']
                circle_instance_vector = circle_process_results['instance_vector']
                circle_size_vector = circle_process_results['size_vector']
                output_circle_shape_plot = circle_process_results['circle_gdf']
                rr_ed_circle = circle_process_results['rr_ed']
                T1_circle = circle_process_results['T1']

                # Update parameters for circle calculations
                params.update({
                    'circle_prediction': circle_prediction.flatten(),
                    'circle_cluster_vector': circle_cluster_vector.flatten(),
                    'circle_instance_vector': circle_instance_vector,
                    'circle_size_vector': circle_size_vector,
                    'output_circle_shape_plot': output_circle_shape_plot,
                    'rr_ed_circle': rr_ed_circle,
                    'T1_circle': T1_circle
                })

                # Calculate circle parameters
                circle_param_results = circle_parameter_cal(params)

                # Extract returned values from the dictionary
                circle_parameter = circle_param_results['circle_parameter']

                # Generate and save circle images
                imaging_circle_updated(ortho_file_path,shape_temp,RESULTS_FOLDER,rr_ed_circle,circle_cluster_vector, circle_prediction)
                # Compile results into DataFrame
                df_t = {
                    'date_id': date_id,
                    'plot_id': plot_id_temp,
                    'plot': P,
                    **leaf_parameter,  # Merge leaf parameters
                    **circle_parameter,  # Merge circle parameters directly
                    'DECISION': DECISION_PLOT_temp
                }
                values = pd.concat([values, pd.DataFrame([df_t])], ignore_index=True)
        
      

# Final processing and saving results
values = pd.DataFrame(values)
values = values.sort_values(by=['plot']).reset_index(drop=True)

# Update spatial data
modified_shape_plot = shape_plot.copy()
modified_shape_plot['DECISION'] = values['DECISION']

# Match design data with values
design_miniplot_mod = design_miniplot.sort_values(by=['plot_id'])
values['grid_id'] = design_miniplot_mod['grid_id'].values
values['block'] = design_miniplot_mod['block'].values
values['miniplot_label'] = design_miniplot_mod['fid'].values
values['fid'] = design_miniplot_mod['fid'].values

# Save parameters to CSV
file_name_parameter_series = "parameter_report.csv"
values.to_csv(os.path.join(RESULTS_FOLDER_ROOT, file_name_parameter_series), index=False)
print(f"Parameters saved to {RESULTS_FOLDER_ROOT / file_name_parameter_series}")

# Save spatial data to GeoPackage
modified_shape_plot_gdf = gpd.GeoDataFrame(modified_shape_plot)
modified_shape_plot_gdf.to_file(os.path.join(RESULTS_FOLDER_ROOT, "application_map.gpkg"), driver="GPKG")
print(f"Spatial data saved to {RESULTS_FOLDER_ROOT / 'application_map.gpkg'}")

# Merging parameters with spatial data
modified_shape_plot['leaf_mean_n_cluster'] = values['leaf_mean_n_cluster']
# Repeat for other parameters as needed

# Save complete spatial data to another GeoPackage
modified_shape_plot_gdf_complete = gpd.GeoDataFrame(modified_shape_plot)
modified_shape_plot_gdf_complete.to_file(os.path.join(RESULTS_FOLDER_ROOT, "application_map_complete.gpkg"), driver="GPKG")
print(f"Complete spatial data saved to {RESULTS_FOLDER_ROOT / 'application_map_complete.gpkg'}")

# Print execution time
end_time = pd.Timestamp.now()
print(f"Execution finished at: {end_time}")
